In [2]:
import pandas as pd

In [3]:
rot_tom_df = pd.read_csv("rotten_tomatoes_movies.csv")

rot_tom_df.head()



,rotten_tomatoes_link,movie_title,movie_info,critics_consensus,content_rating,genres,directors,authors,actors,original_release_date,...,production_company,tomatometer_status,tomatometer_rating,tomatometer_count,audience_status,audience_rating,audience_count,tomatometer_top_critics_count,tomatometer_fresh_critics_count,tomatometer_rotten_critics_count
0,m/0814255,Percy Jackson & the Olympians: The Lightning T...,"Always trouble-prone, the life of teenager Per...",Though it may seem like just another Harry Pot...,PG,"Action & Adventure, Comedy, Drama, Science Fic...",Chris Columbus,"Craig Titley, Chris Columbus, Rick Riordan","Logan Lerman, Brandon T. Jackson, Alexandra Da...",2010-02-12,...,20th Century Fox,Rotten,49.0,149.0,Spilled,53.0,254421.0,43,73,76
1,m/0878835,Please Give,Kate (Catherine Keener) and her husband Alex (...,Nicole Holofcener's newest might seem slight i...,R,Comedy,Nicole Holofcener,Nicole Holofcener,"Catherine Keener, Amanda Peet, Oliver Platt, R...",2010-04-30,...,Sony Pictures Classics,Certified-Fresh,87.0,142.0,Upright,64.0,11574.0,44,123,19
2,m/10,10,"A successful, middle-aged Hollywood songwriter...",Blake Edwards' bawdy comedy may not score a pe...,R,"Comedy, Romance",Blake Edwards,Blake Edwards,"Dudley Moore, Bo Derek, Julie Andrews, Robert ...",1979-10-05,...,Waner Bros.,Fresh,67.0,24.0,Spilled,53.0,14684.0,2,16,8
3,m/1000013-12_angry_men,12 Angry Men (Twelve Angry Men),Following the closing arguments in a murder tr...,Sidney Lumet's feature debut is a superbly wri...,NR,"Classics, Drama",Sidney Lumet,Reginald Rose,"Martin Balsam, John Fiedler, Lee J. Cobb, E.G....",1957-04-13,...,Criterion Collection,Certified-Fresh,100.0,54.0,Upright,97.0,105386.0,6,54,0
4,m/1000079-20000_leagues_under_the_sea,"20,000 Leagues Under The Sea","In 1866, Professor Pierre M. Aronnax (Paul Luk...","One of Disney's finest live-action adventures,...",G,"Action & Adventure, Drama, Kids & Family",Richard Fleischer,Earl Felton,"James Mason, Kirk Douglas, Paul Lukas, Peter L...",1954-01-01,...,Disney,Fresh,89.0,27.0,Upright,74.0,68918.0,5,24,3


In [4]:
imsdb_df = pd.read_csv("imsdb_movies.csv", header=None, names=range(8))

imsdb_df["people"] = imsdb_df.iloc[:, 1:].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)
imsdb_df.rename(columns={0: 'movie_title'}, inplace=True)
imsdb_df.head()

,movie_title,1,2,3,4,5,6,7,people
0,10 Things I Hate About You,Karen McCullah Lutz,Kirsten Smith,William Shakespeare,NaN,NaN,NaN,NaN,"Karen McCullah Lutz,Kirsten Smith,William Shak..."
1,12,Lawrence Bridges,NaN,NaN,NaN,NaN,NaN,NaN,Lawrence Bridges
2,12 and Holding,Anthony Cipriano,NaN,NaN,NaN,NaN,NaN,NaN,Anthony Cipriano
3,12 Monkeys,David Peoples,Janet Peoples,NaN,NaN,NaN,NaN,NaN,"David Peoples,Janet Peoples"
4,12 Years a Slave,John Ridley,NaN,NaN,NaN,NaN,NaN,NaN,John Ridley


In [28]:
imsdb_df['lowercase_movie_title'] = imsdb_df['movie_title'].str.lower()
rot_tom_df['lowercase_movie_title'] = rot_tom_df['movie_title'].str.lower()
rot_tom_df['lowercase_movie_title'] = rot_tom_df['lowercase_movie_title'].str.replace('^The ', '', case=False)
rot_tom_df['lowercase_movie_title'] = rot_tom_df['lowercase_movie_title'].str.replace(' - ', ': ', case=False)


combined_df = imsdb_df.merge(rot_tom_df, how = "left", on = "lowercase_movie_title")
combined_df.head()
combined_df["rotten_tomatoes_link"].notna().sum()


combined_to_csv = combined_df[["movie_title_x", "tomatometer_rating", "audience_rating"]]
combined_to_csv = combined_to_csv.replace(' ', '-', regex=True)
combined_to_csv.head()
combined_to_csv.to_csv("combined_to_csv.csv")
combined_to_csv = combined_to_csv.rename(columns= {"movie_title_x" : "IMSDB_Title", "tomatometer_rating" : "CriticScore", "audience_rating" : "AudienceScore"})
combined_to_csv["RT_Title"] = combined_to_csv["IMSDB_Title"]

unmatched = pd.read_csv("rt_scores.csv")
unmatched = unmatched.replace(' ', '-', regex=True)

print(unmatched.shape)
print(combined_to_csv.shape)
all = pd.concat([unmatched, combined_to_csv])
print(all.shape)
all.to_csv("all_rt_scores.csv")


(196, 4)
(1339, 4)
(1535, 4)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  This is separate from the ipykernel package so we can avoid doing imports until


In [6]:
unmatched_df = combined_df[combined_df["rotten_tomatoes_link"].isna()]

In [7]:
filtered_unmatched = unmatched_df[['lowercase_movie_title', 'people']]

In [8]:
unmatched_string = filtered_unmatched.astype(str).apply(lambda x: ','.join(x), axis=1)

In [9]:
print(unmatched_string)

33                               abyss, The,James Cameron
41                after school special,David H. Steinberg
51                                    alien 3,Rex Pickett
70                         american madness,Robert Riskin
71         american milkshake,David Andalman,Mariko Munro
                              ...                        
1302     white jazz,Joe Carnahan,Matthew Michael Carnahan
1303                     white ribbon, The,Michael Haneke
1307    who's your daddy,Maria Veltre Druse,Jack Sekowski
1308                            wild at heart,David Lynch
1312    wild things: diamonds in the rough,Andy Hurst,...
Length: 196, dtype: object


In [10]:
# Save the resulting Series to a CSV file
unmatched_string.to_csv('unmatched_imsdb_movies.csv', index=False, header=False)